In [1]:
import pyspark

In [9]:
from pyspark.sql.types import StructField, StringType, StringType

In [18]:
from pyspark.sql.functions import *

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [8]:
df = spark.read.csv('case.csv', header=True, inferSchema=True)

Read the case, department, and source data into their own spark dataframes.

In [10]:
df_dept = spark.read.csv('dept.csv', header=True, inferSchema=True)
df_source = spark.read.csv('source.csv', header=True, inferSchema=True)

In [6]:
#event table
df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: string (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: string (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [11]:
#dimension table
df_dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [12]:
#dimension table
df_source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [13]:
df.write.json("case.json", mode='overwrite')

In [16]:
df.write.json("dept.json", mode='overwrite')
df.write.json("source.json", mode='overwrite')

In [14]:
blob = spark.read.format('json').load('case.json/')

In [15]:
blob.printSchema()

root
 |-- SLA_days: double (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- case_id: long (nullable = true)
 |-- case_late: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_status: string (nullable = true)
 |-- council_district: long (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- request_address: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- source_id: string (nullable = true)



Inspect your folder structure. What do you notice?

    Those JSONs are a directory rather than one JSON file. I tried reading from it and it came out the way I wrote. So I'll jsut have to remember that sparkDF.write.json makes a directory.

Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [19]:
#The withColumn makes a new column. Since case_closed existed before we are mutating it. It's now a boolean value where
#before it was a string. 
df = df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "yes"')
)

In [22]:
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [21]:
#Made a new column that takes the place of the old one. 
#.withcolumn looks like it takes two parameters: 1 is the name of the new column
#                                               2 is the terms of what this new column will be
df = df.withColumn('council_district', col('council_district').cast("string"))
#df = df.withColumn("council_district", col("council_district").cast("string"))

How old is the latest (in terms of days past SLA) currently open issue? 

How long has the oldest (in terms of days since opened) currently opened issue been open?

How many Stray Animal cases are there?

How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

Convert the council_district column to a string column.

Extract the year from the case_closed_date column.

Convert num_days_late from days to hours in new columns num_hours_late.

Join the case data with the source and department data.

Are there any cases that do not have a request source?

What are the top 10 service request types in terms of number of requests?

What are the top 10 service request types in terms of average days late?

Does number of days late depend on department?

How do number of days late depend on department and request type?